In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)
pd.set_option('display.float_format',lambda x : '%.2f' % x)
import numpy as np
np.set_printoptions(suppress=True)
from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

import lightgbm as lgb

In [2]:
t_base_info = "./dataset/train/base_info.csv"
t_annual_report_info = "./dataset/train/annual_report_info.csv"
t_tax_info = "./dataset/train/tax_info.csv"
t_change_info = "./dataset/train/change_info.csv"
t_news_info = "./dataset/train/news_info.csv"
t_other_info = "./dataset/train/other_info.csv"
t_entprise_info = "./dataset/train/entprise_info.csv" # train labeled data

In [3]:
df_tax_info = pd.read_csv(t_tax_info, header=0)
print("tax info: \n", df_tax_info)
print("tax info: \n", df_tax_info.shape)

tax info: 
                                                      id  START_DATE  \
0      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
1      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
2      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
3      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
4      f000950527a6feb6c2f40c9d8477e73a439dfa0897830397  2015/09/01   
...                                                 ...         ...   
29190  f000950527a6feb6cb8976eb56233ede461cb23103f85f32  2015/12/08   
29191  f000950527a6feb6cb8976eb56233ede461cb23103f85f32  2015/12/08   
29192  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   
29193  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   
29194  d8071a739aa75a3bbb9e08ebd134ae1289f194b70cac0e95  2016/07/01   

         END_DATE TAX_CATEGORIES     TAX_ITEMS  TAXATION_BASIS  TAX_RATE  \
0      2015/09/30            印花税      工伤保险（单位）        72530

In [4]:
## 提取天数差
from util import *
df_tax_info['TAX_DAYS'] = df_tax_info[['START_DATE', 'END_DATE']].apply(lambda x : days_v1(x['END_DATE'], x['START_DATE']), axis=1)

del df_tax_info['START_DATE']
del df_tax_info['END_DATE']

print(df_tax_info.dtypes)
print(df_tax_info.head())


id                 object
TAX_CATEGORIES     object
TAX_ITEMS          object
TAXATION_BASIS    float64
TAX_RATE          float64
DEDUCTION         float64
TAX_AMOUNT        float64
TAX_DAYS            int64
dtype: object
                                                 id TAX_CATEGORIES  \
0  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
1  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
2  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
3  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   
4  f000950527a6feb6c2f40c9d8477e73a439dfa0897830397            印花税   

      TAX_ITEMS  TAXATION_BASIS  TAX_RATE  DEDUCTION  TAX_AMOUNT  TAX_DAYS  
0      工伤保险（单位）        72530.75      0.00      -0.04       21.80        29  
1      失业保险（单位）        72530.75      0.00      -0.04       21.80        29  
2      医疗保险（单位）        72530.75      0.00      -0.04       21.80        29  
3  企业养老保险基金（单位）        72530.75      0.00      -0

In [18]:
df_tax_info_cp = df_tax_info.copy()
df_tax_amount_cp = df_tax_info_cp[['id', 'TAX_DAYS', 'TAX_RATE', 'DEDUCTION', 'TAX_AMOUNT', 'TAXATION_BASIS']].groupby(['id'], as_index=False).sum()

# 加上平均
df_tax_amount_cp['arg_tax_rate'] =  df_tax_amount_cp['TAX_RATE'] / df_tax_amount_cp['TAX_DAYS']
df_tax_amount_cp['arg_tax_deduction'] =  df_tax_amount_cp['DEDUCTION'] / df_tax_amount_cp['TAX_DAYS']
df_tax_amount_cp['arg_tax_amount'] =  df_tax_amount_cp['TAX_AMOUNT'] / df_tax_amount_cp['TAX_DAYS']
df_tax_amount_cp['arg_tax_basis'] =  df_tax_amount_cp['TAXATION_BASIS'] / df_tax_amount_cp['TAX_DAYS']

del df_tax_amount_cp['TAX_DAYS']

print(df_tax_amount_cp.head(5))
print(df_tax_amount_cp.shape)

                                                 id  TAX_RATE  DEDUCTION  \
0  216bd2aaf4d079243f3c0bd3d6d28333c790bd3aee0ddad8      1.45       0.60   
1  216bd2aaf4d079248a1cb9c41425810a25d29c1fc1d1c15a      0.00       0.00   
2  216bd2aaf4d07924caa4f30fb76969cba69358e90e310f5e      0.00       0.00   
3  216bd2aaf4d07924d90a8951ef6a87c01e2f2ec78aa14698      0.00       0.00   
4  47645761dc56bb8c0ee994e2cf201d523319b7160f6722f8      0.00       0.00   

   TAX_AMOUNT  TAXATION_BASIS  arg_tax_rate  arg_tax_deduction  \
0     1727.89       894428.56          0.00               0.00   
1    25038.50            0.00          0.00               0.00   
2     2400.00            0.00          0.00               0.00   
3      204.60            0.00          0.00               0.00   
4       73.94            0.00          0.00               0.00   

   arg_tax_amount  arg_tax_basis  
0            1.30         670.49  
1          208.65           0.00  
2           80.00           0.00  
3     

In [5]:
# one-hot
df_tax_item = pd.get_dummies(df_tax_info['TAX_ITEMS'], prefix='tax_items')
df_tax_cate = pd.get_dummies(df_tax_info['TAX_CATEGORIES'], prefix='tax_categories')

df_tax_info = pd.concat([df_tax_info, df_tax_item, df_tax_cate], axis=1)

del df_tax_info['TAX_ITEMS']
del df_tax_info['TAX_CATEGORIES']

print(df_tax_info.head())
print(df_tax_info.shape)


                   0                 0   
3                     0                     0                 0   
4                     0                     0                 0   

   tax_items_失业保险滞纳金  tax_items_失业保险违章罚款  tax_items_失业保险（单位）  \
0                  0                   0                   0   
1                  0                   0                   1   
2                  0                   0                   0   
3                  0                   0                   0   
4                  0                   0                   0   

   tax_items_失地农民养老保险费  tax_items_契税  tax_items_娱乐业  tax_items_安装  \
0                    0             0              0             0   
1                    0             0              0             0   
2                    0             0              0             0   
3                    0             0              0             0   
4                    0             0              0             0   

   tax_items_对企事业单位的承包、

In [6]:
print(df_tax_info.columns)
print("items: \n", df_tax_item.columns)
print("categories: \n", df_tax_cate.columns)

Index(['id', 'TAXATION_BASIS', 'TAX_RATE', 'DEDUCTION', 'TAX_AMOUNT',
       'TAX_DAYS', 'tax_items_10吨以下非机动船', 'tax_items_150吨以下机动船',
       'tax_items_20人以上汽车及2.2升以上小轿车', 'tax_items_20人以下乘人汽车',
       ...
       'tax_categories_城市维护建设税', 'tax_categories_城镇土地使用税', 'tax_categories_契税',
       'tax_categories_房产税', 'tax_categories_教育费附加', 'tax_categories_残疾人就业保障金',
       'tax_categories_水利建设专项收入', 'tax_categories_税务部门罚没收入',
       'tax_categories_耕地占用税', 'tax_categories_营业税'],
      dtype='object', length=298)
items: 
 Index(['tax_items_10吨以下非机动船', 'tax_items_150吨以下机动船',
       'tax_items_20人以上汽车及2.2升以上小轿车', 'tax_items_20人以下乘人汽车', 'tax_items_一等',
       'tax_items_一等（县区）', 'tax_items_一类地区', 'tax_items_一类地区（占用基本农田）',
       'tax_items_一般房屋出租', 'tax_items_三等',
       ...
       'tax_items_销售建筑物或构筑物', 'tax_items_销售额或营业额', 'tax_items_非普通住宅（预征）',
       'tax_items_非机动车', 'tax_items_面积（旅店业、餐饮业、娱乐业）', 'tax_items_预提所得税',
       'tax_items_飞机场跑道、停机坪', 'tax_items_餐饮、住宿、娱乐营业额随征', 'tax_items_餐饮业',

      nan   
41       nan        nan         nan       nan             nan       nan   
62       nan        nan         nan       nan             nan       nan   

    DEDUCTION  TAX_AMOUNT  TAX_DAYS  TAXATION_BASIS  TAX_RATE  DEDUCTION  \
0         nan         nan       nan             nan       nan        nan   
30        nan         nan       nan             nan       nan        nan   
40        nan         nan       nan             nan       nan        nan   
41        nan         nan       nan             nan       nan        nan   
62        nan         nan       nan             nan       nan        nan   

    TAX_AMOUNT  TAX_DAYS  TAXATION_BASIS  TAX_RATE  DEDUCTION  TAX_AMOUNT  \
0          nan       nan             nan       nan        nan         nan   
30         nan       nan             nan       nan        nan         nan   
40         nan       nan             nan       nan        nan         nan   
41         nan       nan             nan       nan        nan         n

In [8]:
for col in tqdm(df_tax.columns):
    print(col, df_tax[col].nunique(dropna=False))

75, dtype: int64
TAX_RATE TAX_RATE    221
TAX_RATE    221
TAX_RATE    221
TAX_RATE    221
TAX_RATE    221
           ... 
TAX_RATE    221
TAX_RATE    221
TAX_RATE    221
TAX_RATE    221
TAX_RATE    221
Length: 275, dtype: int64
DEDUCTION DEDUCTION    182
DEDUCTION    182
DEDUCTION    182
DEDUCTION    182
DEDUCTION    182
            ... 
DEDUCTION    182
DEDUCTION    182
DEDUCTION    182
DEDUCTION    182
DEDUCTION    182
Length: 275, dtype: int64
TAX_AMOUNT TAX_AMOUNT    790
TAX_AMOUNT    790
TAX_AMOUNT    790
TAX_AMOUNT    790
TAX_AMOUNT    790
             ... 
TAX_AMOUNT    790
TAX_AMOUNT    790
TAX_AMOUNT    790
TAX_AMOUNT    790
TAX_AMOUNT    790
Length: 275, dtype: int64
TAX_DAYS TAX_DAYS    249
TAX_DAYS    249
TAX_DAYS    249
TAX_DAYS    249
TAX_DAYS    249
           ... 
TAX_DAYS    249
TAX_DAYS    249
TAX_DAYS    249
TAX_DAYS    249
TAX_DAYS    249
Length: 275, dtype: int64
TAXATION_BASIS TAXATION_BASIS    383
TAXATION_BASIS    383
TAXATION_BASIS    383
TAXATION_BASIS    383


KeyboardInterrupt: 